In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier
from tqdm import tqdm
from sklearn.metrics import f1_score ,roc_auc_score
import time
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

import lightgbm as lgb

In [12]:
path = '/Data2/folder_ti/data/'
train_csv = '1st_1011_train.csv'

train = pd.read_csv(path+train_csv,index_col=0)
train = train.astype({'gender':'object'})


In [4]:
path = '/Data2/folder_ti/data/'
second_csv = '2nd_classification_train.csv'

second = pd.read_csv(path+second_csv,index_col=0)

/opt/conda/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [13]:
train.head()

,application_id,user_id,purpose,is_applied,past.is_applied,m_past.is_applied,seq,min.loan_rate,product_n,n.limit_over_desire,...,employment_type,houseown_type,desired_amount,rehabilitation,existing_loan_cnt,logdata,diff,diff2,limit_over_desire_product,yd
83603,248576,9,사업자금,노대출,-1,0.465641,0.0,4.3,26,15,...,계약직,기타가족소유,15000000.0,2,1,3,0.0,0.000000,0.807692,0.517241
396875,1180616,11,생활비,대출,-1,0.465641,0.0,10.0,25,15,...,정규직,기타가족소유,2000000.0,-1,1,12,-1000000.0,-0.500000,0.960000,0.080000
588684,1751522,11,대환대출,대출,1,1.000000,1.0,10.7,25,15,...,정규직,기타가족소유,13000000.0,-1,1,21,31000000.0,2.384615,0.680000,0.520000
507188,1508745,14,대환대출,노대출,-1,0.465641,0.0,13.7,2,0,...,정규직,전월세,30000000.0,-1,0,8,-3000000.0,-0.100000,0.000000,1.111111
669142,1990215,17,생활비,대출,-1,0.465641,0.0,17.9,2,2,...,계약직,기타가족소유,3000000.0,-1,0,20,26000000.0,2.835714,1.000000,0.060000


In [7]:
second.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,age,gender,...,houseown_type,desired_amount,purpose,rehabilitation,existing_loan_cnt,rate_rank,diff,rate_diff_with.min,rate_diff_with.above,logdata
1,1,2022-05-06 11:05:57,62,200,8000000.0,14.8,0,777860,33,1,...,������,30000000.0,��ȯ����,2,0.0,1.0,-22000000.0,0.0,0.00,7
2,1,2022-05-06 11:05:43,38,223,19000000.0,17.1,0,777860,33,1,...,������,30000000.0,��ȯ����,2,0.0,2.5,-11000000.0,2.3,1.35,7
3,1,2022-05-06 11:05:56,38,16,19000000.0,17.1,0,777860,33,1,...,������,30000000.0,��ȯ����,2,0.0,2.5,-11000000.0,2.3,0.00,7
4,1,2022-05-06 11:05:42,59,150,19000000.0,17.7,0,777860,33,1,...,������,30000000.0,��ȯ����,2,0.0,4.0,-11000000.0,2.9,0.60,7
5,1,2022-05-06 11:05:44,37,206,32000000.0,18.9,0,777860,33,1,...,������,30000000.0,��ȯ����,2,0.0,5.0,2000000.0,4.1,1.20,7


In [8]:
second.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10264386 entries, 1 to 13519868
Data columns (total 26 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   application_id         int64  
 1   loanapply_insert_time  object 
 2   bank_id                int64  
 3   product_id             int64  
 4   loan_limit             float64
 5   loan_rate              float64
 6   is_applied             int64  
 7   user_id                int64  
 8   age                    int64  
 9   gender                 int64  
 10  insert_time            object 
 11  credit_score           int64  
 12  yearly_income          float64
 13  income_type            object 
 14  company_year           int64  
 15  employment_type        object 
 16  houseown_type          object 
 17  desired_amount         float64
 18  purpose                object 
 19  rehabilitation         int64  
 20  existing_loan_cnt      float64
 21  rate_rank              float64
 22  diff            

In [9]:
second.nunique()

application_id            728862
loanapply_insert_time    2425396
bank_id                       62
product_id                   168
loan_limit                    70
loan_rate                    173
is_applied                     2
user_id                   259321
age                           73
gender                         3
insert_time               681142
credit_score                  81
yearly_income                 83
income_type                    6
company_year                  12
employment_type                4
houseown_type                  4
desired_amount               111
purpose                        8
rehabilitation                 4
existing_loan_cnt             10
rate_rank                    155
diff                        3272
rate_diff_with.min           195
rate_diff_with.above          41
logdata                       35
dtype: int64

In [14]:
for col in train.columns :
    print('col name & nan : ', col,'/',train[col].isnull().sum())

col name & nan :  application_id / 0
col name & nan :  user_id / 0
col name & nan :  purpose / 0
col name & nan :  is_applied / 0
col name & nan :  past.is_applied / 0
col name & nan :  m_past.is_applied / 0
col name & nan :  seq / 0
col name & nan :  min.loan_rate / 0
col name & nan :  product_n / 0
col name & nan :  n.limit_over_desire / 0
col name & nan :  loan_limit_min.rate / 0
col name & nan :  age / 0
col name & nan :  gender / 0
col name & nan :  credit_score / 0
col name & nan :  yearly_income / 0
col name & nan :  income_type / 0
col name & nan :  company_year / 0
col name & nan :  employment_type / 0
col name & nan :  houseown_type / 0
col name & nan :  desired_amount / 0
col name & nan :  rehabilitation / 0
col name & nan :  existing_loan_cnt / 0
col name & nan :  logdata / 0
col name & nan :  diff / 0
col name & nan :  diff2 / 0
col name & nan :  limit_over_desire_product / 0
col name & nan :  yd / 0


In [19]:
common_drop_col = ['user_id','age','gender','credit_score','yearly_income','income_type','company_year','employment_type',
                  'houseown_type','desired_amount','purpose','rehabilitation','existing_loan_cnt','diff','logdata']
second_drop_col = ['loanapply_insert_time','insert_time']


In [20]:
for col in second.columns :
    print('col name & nan : ', col,'/',second[col].isnull().sum())

col name & nan :  application_id / 0
col name & nan :  loanapply_insert_time / 0
col name & nan :  bank_id / 0
col name & nan :  product_id / 0
col name & nan :  loan_limit / 0
col name & nan :  loan_rate / 0
col name & nan :  is_applied / 0
col name & nan :  user_id / 0
col name & nan :  age / 0
col name & nan :  gender / 0
col name & nan :  insert_time / 0
col name & nan :  credit_score / 0
col name & nan :  yearly_income / 0
col name & nan :  income_type / 0
col name & nan :  company_year / 0
col name & nan :  employment_type / 0
col name & nan :  houseown_type / 0
col name & nan :  desired_amount / 0
col name & nan :  purpose / 0
col name & nan :  rehabilitation / 0
col name & nan :  existing_loan_cnt / 0
col name & nan :  rate_rank / 0
col name & nan :  diff / 0
col name & nan :  rate_diff_with.min / 0
col name & nan :  rate_diff_with.above / 0
col name & nan :  logdata / 0


In [21]:
second_ = second
second.drop(common_drop_col+second_drop_col,axis=1,inplace=True)

In [22]:
second.head()

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,rate_rank,rate_diff_with.min,rate_diff_with.above
1,1,62,200,8000000.0,14.8,0,1.0,0.0,0.00
2,1,38,223,19000000.0,17.1,0,2.5,2.3,1.35
3,1,38,16,19000000.0,17.1,0,2.5,2.3,0.00
4,1,59,150,19000000.0,17.7,0,4.0,2.9,0.60
5,1,37,206,32000000.0,18.9,0,5.0,4.1,1.20


In [23]:
cat = ['purpose','income_type','employment_type','houseown_type']
train2 = pd.get_dummies(train, columns = cat)
train2.head()

,application_id,user_id,is_applied,past.is_applied,m_past.is_applied,seq,min.loan_rate,product_n,n.limit_over_desire,loan_limit_min.rate,...,income_type_PRACTITIONER,income_type_PRIVATEBUSINESS,employment_type_계약직,employment_type_기타,employment_type_일용직,employment_type_정규직,houseown_type_기타가족소유,houseown_type_배우자,houseown_type_자가,houseown_type_전월세
83603,248576,9,노대출,-1,0.465641,0.0,4.3,26,15,15000000.0,...,0,0,1,0,0,0,1,0,0,0
396875,1180616,11,대출,-1,0.465641,0.0,10.0,25,15,1000000.0,...,0,0,0,0,0,1,1,0,0,0
588684,1751522,11,대출,1,1.000000,1.0,10.7,25,15,42500000.0,...,0,0,0,0,0,1,1,0,0,0
507188,1508745,14,노대출,-1,0.465641,0.0,13.7,2,0,27000000.0,...,0,0,0,0,0,1,0,0,0,1
669142,1990215,17,대출,-1,0.465641,0.0,17.9,2,2,29000000.0,...,0,0,1,0,0,0,1,0,0,0


In [6]:
train2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 728884 entries, 83604 to 518933
Data columns (total 32 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   application_id               728884 non-null  int64  
 1   last.loan_insert.time        728884 non-null  object 
 2   user_id                      728884 non-null  int64  
 3   is_applied                   728884 non-null  int64  
 4   past.is_applied              728884 non-null  int64  
 5   m_past.is_applied            728884 non-null  float64
 6   seq                          728884 non-null  int64  
 7   age                          728884 non-null  int64  
 8   gender                       728884 non-null  object 
 9   credit_score                 728884 non-null  int64  
 10  yearly_income                728884 non-null  float64
 11  company_year                 728884 non-null  int64  
 12  employment_type              728884 non-null  object 


In [24]:
train.nunique()

application_id               728862
user_id                      259321
purpose                           8
is_applied                        2
past.is_applied                   3
m_past.is_applied              1752
seq                              14
min.loan_rate                   173
product_n                        47
n.limit_over_desire              16
loan_limit_min.rate              44
age                              73
gender                            3
credit_score                     81
yearly_income                    87
income_type                       6
company_year                     11
employment_type                   4
houseown_type                     4
desired_amount                   66
rehabilitation                    4
existing_loan_cnt                12
logdata                          35
diff                            208
diff2                          2003
limit_over_desire_product      1662
yd                             2307
dtype: int64

In [25]:
train.count()

application_id               728862
user_id                      728862
purpose                      728862
is_applied                   728862
past.is_applied              728862
m_past.is_applied            728862
seq                          728862
min.loan_rate                728862
product_n                    728862
n.limit_over_desire          728862
loan_limit_min.rate          728862
age                          728862
gender                       728862
credit_score                 728862
yearly_income                728862
income_type                  728862
company_year                 728862
employment_type              728862
houseown_type                728862
desired_amount               728862
rehabilitation               728862
existing_loan_cnt            728862
logdata                      728862
diff                         728862
diff2                        728862
limit_over_desire_product    728862
yd                           728862
dtype: int64

In [27]:
def trans(x):
    if x == '노대출':
        return 0
    elif x == '대출':
        return 1
train['is_applied'] = train['is_applied'].apply(trans)

In [28]:
train.is_applied.value_counts()

1    377534
0    351328
Name: is_applied, dtype: int64

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 728884 entries, 83604 to 518933
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   application_id         728884 non-null  int64  
 1   last.loan_insert.time  728884 non-null  object 
 2   user_id                728884 non-null  int64  
 3   is_applied             728884 non-null  int64  
 4   purpose                728884 non-null  object 
 5   past.is_applied        728884 non-null  int64  
 6   m_past.is_applied      728884 non-null  float64
 7   seq                    728884 non-null  int64  
 8   age                    728884 non-null  int64  
 9   gender                 728884 non-null  object 
 10  credit_score           728884 non-null  int64  
 11  yearly_income          728884 non-null  float64
 12  income_type            728884 non-null  object 
 13  company_year           728884 non-null  int64  
 14  employment_type        728884 no

In [29]:
data_0 = train[train.is_applied == 0]
data_1 = train[train.is_applied == 1]
target = 'is_applied'    
if "set" not in data_0.columns:
    data_0["set"] = np.random.choice(["train", "valid"], p =[.5, .5], size=(data_0.shape[0],))
if "set" not in data_1.columns:
    data_1["set"] = np.random.choice(["train", "valid"], p =[.5, .5], size=(data_1.shape[0],))

/tmp/ipykernel_83489/108332005.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_0["set"] = np.random.choice(["train", "valid"], p =[.5, .5], size=(data_0.shape[0],))
/tmp/ipykernel_83489/108332005.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1["set"] = np.random.choice(["train", "valid"], p =[.5, .5], size=(data_1.shape[0],))


In [30]:
train = pd.concat([data_0,data_1], ignore_index=True)
train = train.sample(frac=1).reset_index(drop=True)

In [31]:
cat_col = ['purpose','gender','income_type','employment_type','houseown_type','rehabilitation']

In [32]:
cat_trans = dict()
for cat in cat_col :
    cat_trans[cat] = dict()
    type_ = list(set(train[cat].tolist()))
    for i, ty in enumerate(type_) :
        cat_trans[cat][ty] = i

In [33]:
cat_trans

{'purpose': {'전월세보증금': 0,
  '투자': 1,
  '대환대출': 2,
  '사업자금': 3,
  '기타': 4,
  '자동차구입': 5,
  '주택구입': 6,
  '생활비': 7},
 'gender': {0: 0, 1: 1, -1: 2},
 'income_type': {'OTHERINCOME': 0,
  'EARNEDINCOME': 1,
  'EARNEDINCOME2': 2,
  'PRACTITIONER': 3,
  'FREELANCER': 4,
  'PRIVATEBUSINESS': 5},
 'employment_type': {'일용직': 0, '기타': 1, '계약직': 2, '정규직': 3},
 'houseown_type': {'전월세': 0, '배우자': 1, '자가': 2, '기타가족소유': 3},
 'rehabilitation': {0: 0, 1: 1, 2: 2, -1: 3}}

In [34]:
from tqdm import tqdm
for cat in cat_col :
    if cat == 'gender' :
        continue
    for k in tqdm(cat_trans[cat].keys()) :
        train[cat][train[cat]==k] = cat_trans[cat][k]   

  0%|                                                                                                                       | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_83489/4081748353.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[cat][train[cat]==k] = cat_trans[cat][k]
  0%|                                                                                                                       | 0/4 [00:00<?, ?it/s]/tmp/ipykernel_83489/4081748353.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[cat][train[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_83489/4081748353.py:6: SettingWithCopyWarning: 
A value is 

In [35]:
train_indices = train[train.set=="train"].index
valid_indices = train[train.set=="valid"].index

In [36]:
types = train.dtypes
target = 'is_applied'
unused_col = ['set','user_id','application_id']
cat_col = ['purpose','income_type','rehabilitation','gender','employment_type','houseown_type']

features = [ col for col in train.columns if col not in unused_col+[target]] 

In [37]:
#nunique = train.nunique()
types = train.dtypes
target = 'is_applied'
unused_col = ['set','user_id','application_id']
cat_col = ['purpose','income_type','rehabilitation','gender','employment_type','houseown_type']
categorical_dims =  {}
for col in train.columns:
    if col in cat_col :
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_dims[col] = len(l_enc.classes_)
    elif col not in unused_col+[target]:
        train.fillna(train.loc[:, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
features = [ col for col in train.columns if col not in unused_col+[target]] 
cat_idxs = [ i for i, f in enumerate(features) if f in cat_col]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in cat_col]
#train_shuf = train_shuf.astype({'income_type':'int64','employment_type':'int64','houseown_type':'int64','purpose':'int64'})

purpose 8
gender 3
income_type 6
employment_type 4
houseown_type 4
rehabilitation 4


In [38]:
train.head()

,application_id,user_id,purpose,is_applied,past.is_applied,m_past.is_applied,seq,min.loan_rate,product_n,n.limit_over_desire,...,houseown_type,desired_amount,rehabilitation,existing_loan_cnt,logdata,diff,diff2,limit_over_desire_product,yd,set
0,1345430,409465,7,1,1,0.666667,3.0,9.2,4,3,...,0,8000000.0,3,1,0,13000000.0,1.625000,0.750000,0.266667,train
1,1340543,789104,7,1,1,1.000000,4.0,11.1,2,2,...,0,15000000.0,3,1,17,2000000.0,0.133333,1.000000,0.600000,train
2,1519681,576002,2,1,0,0.000000,1.0,15.4,5,0,...,2,64500000.0,3,8,8,-49500000.0,-0.767442,0.000000,1.131579,train
3,1488854,30828,7,0,1,0.750000,4.0,6.7,3,0,...,0,30000000.0,3,6,8,-22000000.0,-0.733333,0.000000,0.545455,valid
4,415064,398837,7,0,-1,0.465641,0.0,6.7,47,15,...,2,10000000.0,3,2,11,-1000000.0,-0.100000,0.716667,0.303030,valid


In [39]:
features

['purpose',
 'past.is_applied',
 'm_past.is_applied',
 'seq',
 'min.loan_rate',
 'product_n',
 'n.limit_over_desire',
 'loan_limit_min.rate',
 'age',
 'gender',
 'credit_score',
 'yearly_income',
 'income_type',
 'company_year',
 'employment_type',
 'houseown_type',
 'desired_amount',
 'rehabilitation',
 'existing_loan_cnt',
 'logdata',
 'diff',
 'diff2',
 'limit_over_desire_product',
 'yd']

In [40]:

X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

In [41]:
import torch
DEVICE = 'cuda:2' if torch.cuda.is_available() else 'cpu'
print("Using {}".format(DEVICE))

Using cuda:2


In [42]:

params = {'n_d' : 4,
         'n_steps' :5,        # tree에 depth
         'gamma':1.5,
         'n_independent':2,
          'cat_idxs':cat_idxs,
          'cat_dims':cat_dims,
         'n_shared':2,
         'cat_emb_dim':1,
         'lambda_sparse':1e-4,
         'momentum':0.5,
         'clip_value':2.,
         'optimizer_fn':torch.optim.Adam,
         'optimizer_params':dict(lr=2e-2),
         'scheduler_params':{"gamma": 0.95,
                         "step_size": 20},
         'scheduler_fn':torch.optim.lr_scheduler.StepLR,
         'epsilon':1e-15,
         'device_name':DEVICE,
         'mask_type':'sparsemax'}

clf = TabNetClassifier(**params
)
s = time.time()

clf.fit(X_train=X_train, y_train=y_train,eval_set=[(X_valid, y_valid)],
    max_epochs=50 , patience=20,
    batch_size=10240, virtual_batch_size=1280,
    num_workers=0,
    weights=1,
    drop_last=False
)

print('TabNet gpu sec : ',time.time()- s)


pred_valid = clf.predict(X_valid)


f1_valid = f1_score(y_valid, pred_valid) # y_true : 정답 값, y_pred : 예측 값 
print("Valid F1 score: {}".format(f1_valid))
 

from sklearn.metrics import roc_auc_score 
roc_auc = roc_auc_score(y_valid, pred_valid)
print("Valid ROC AUC Score: {}".format(roc_auc))

/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda:2
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.69555 | val_0_auc: 0.60529 |  0:00:09s
epoch 1  | loss: 0.64783 | val_0_auc: 0.63929 |  0:00:19s
epoch 2  | loss: 0.64065 | val_0_auc: 0.65271 |  0:00:28s
epoch 3  | loss: 0.63956 | val_0_auc: 0.66642 |  0:00:37s
epoch 4  | loss: 0.63861 | val_0_auc: 0.68564 |  0:00:46s
epoch 5  | loss: 0.63506 | val_0_auc: 0.68872 |  0:00:55s
epoch 6  | loss: 0.63393 | val_0_auc: 0.69304 |  0:01:06s
epoch 7  | loss: 0.6327  | val_0_auc: 0.69149 |  0:01:16s
epoch 8  | loss: 0.63089 | val_0_auc: 0.69736 |  0:01:25s
epoch 9  | loss: 0.62827 | val_0_auc: 0.70101 |  0:01:35s
epoch 10 | loss: 0.62694 | val_0_auc: 0.7069  |  0:01:45s
epoch 11 | loss: 0.62412 | val_0_auc: 0.70868 |  0:01:55s
epoch 12 | loss: 0.62463 | val_0_auc: 0.71013 |  0:02:04s
epoch 13 | loss: 0.62281 | val_0_auc: 0.70966 |  0:04:39s
epoch 14 | loss: 0.6208  | val_0_auc: 0.71274 |  0:04:49s
epoch 15 | loss: 0.62159 | val_0_auc: 0.71392 |  0:04:58s
epoch 16 | loss: 0.62094 | val_0_auc: 0.71422 |  0:05:06s
epoch 17 | los

/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TabNet gpu sec :  682.4036786556244
Valid F1 score: 0.6781100225022864
Valid ROC AUC Score: 0.6708019540503026


In [43]:
pred_prob_valid = clf.predict_proba(X_valid)

def thres_optimization(threshold, array) :
    pred = np.ceil(array[:,1]-threshold)
    return pred

score_valid = [0]
best_thres_valid = 0
for i in range(1,100) :
    pre_valid = thres_optimization(i/100,pred_prob_valid)
    
    f1_valid = f1_score(y_valid, pre_valid)
    
    score_valid.append(f1_valid)
    
best_thres_valid = score_valid.index(max(score_valid))/100
print(max(score_valid))
print(best_thres_valid)

    

0.7213720133859187
0.34


# 2nd Classification

In [44]:
pred_valid = thres_optimization(0.34,pred_prob_valid)

In [50]:
valid_set = train.loc[valid_indices,features+['application_id']]
valid_set['pred'] = pred_valid

In [51]:
valid_set.head()

,purpose,past.is_applied,m_past.is_applied,seq,min.loan_rate,product_n,n.limit_over_desire,loan_limit_min.rate,age,gender,...,desired_amount,rehabilitation,existing_loan_cnt,logdata,diff,diff2,limit_over_desire_product,yd,application_id,pred
3,7,1,0.750000,4.0,6.7,3,0,8000000.0,43,2,...,30000000.0,3,6,8,-22000000.0,-0.733333,0.000000,0.545455,1488854,1.0
4,7,-1,0.465641,0.0,6.7,47,15,9000000.0,35,2,...,10000000.0,3,2,11,-1000000.0,-0.100000,0.716667,0.303030,415064,1.0
5,7,1,0.250000,4.0,16.8,2,0,30000000.0,50,2,...,64500000.0,3,5,4,-34500000.0,-0.534884,0.000000,0.750000,240033,-0.0
10,2,-1,0.465641,0.0,6.5,17,0,12000000.0,44,1,...,30000000.0,3,2,17,-18000000.0,-0.600000,0.000000,1.428571,1090213,1.0
11,2,1,0.500000,2.0,12.5,6,6,19000000.0,27,1,...,10000000.0,3,2,0,9000000.0,0.900000,1.000000,0.263158,666684,1.0


In [52]:
second.head()

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,rate_rank,rate_diff_with.min,rate_diff_with.above
1,1,62,200,8000000.0,14.8,0,1.0,0.0,0.00
2,1,38,223,19000000.0,17.1,0,2.5,2.3,1.35
3,1,38,16,19000000.0,17.1,0,2.5,2.3,0.00
4,1,59,150,19000000.0,17.7,0,4.0,2.9,0.60
5,1,37,206,32000000.0,18.9,0,5.0,4.1,1.20


In [53]:
train_2nd = pd.merge(second, valid_set, on='application_id', how='left')
train_2nd.head()

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,rate_rank,rate_diff_with.min,rate_diff_with.above,purpose,...,houseown_type,desired_amount,rehabilitation,existing_loan_cnt,logdata,diff,diff2,limit_over_desire_product,yd,pred
0,1,62,200,8000000.0,14.8,0,1.0,0.0,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,38,223,19000000.0,17.1,0,2.5,2.3,1.35,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,38,16,19000000.0,17.1,0,2.5,2.3,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,59,150,19000000.0,17.7,0,4.0,2.9,0.60,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,37,206,32000000.0,18.9,0,5.0,4.1,1.20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
for col in train_2nd.columns :
    print('col name & nan : ', col,'/',train_2nd[col].isnull().sum())

col name & nan :  application_id / 0
col name & nan :  bank_id / 0
col name & nan :  product_id / 0
col name & nan :  loan_limit / 0
col name & nan :  loan_rate / 0
col name & nan :  is_applied / 0
col name & nan :  rate_rank / 0
col name & nan :  rate_diff_with.min / 0
col name & nan :  rate_diff_with.above / 0
col name & nan :  purpose / 5123908
col name & nan :  past.is_applied / 5123908
col name & nan :  m_past.is_applied / 5123908
col name & nan :  seq / 5123908
col name & nan :  min.loan_rate / 5123908
col name & nan :  product_n / 5123908
col name & nan :  n.limit_over_desire / 5123908
col name & nan :  loan_limit_min.rate / 5123908
col name & nan :  age / 5123908
col name & nan :  gender / 5123908
col name & nan :  credit_score / 5123908
col name & nan :  yearly_income / 5123908
col name & nan :  income_type / 5123908
col name & nan :  company_year / 5123908
col name & nan :  employment_type / 5123908
col name & nan :  houseown_type / 5123908
col name & nan :  desired_amount / 

In [55]:
train_2nd.dropna(axis=0, how='any',inplace=True)

In [69]:
train_2nd.nunique()

application_id               364473
bank_id                          62
product_id                      165
loan_limit                       70
loan_rate                       173
is_applied                        2
rate_rank                       155
rate_diff_with.min              194
rate_diff_with.above             41
purpose                           8
past.is_applied                   3
m_past.is_applied              1307
seq                              14
product_n                        47
n.limit_over_desire              16
age                              72
gender                            3
credit_score                     78
yearly_income                    87
income_type                       6
company_year                     11
employment_type                   4
houseown_type                     4
desired_amount                   66
rehabilitation                    4
existing_loan_cnt                12
logdata                          35
diff                        

In [62]:
train_2nd.iloc[:,:20].head()

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,rate_rank,rate_diff_with.min,rate_diff_with.above,purpose,past.is_applied,m_past.is_applied,seq,min.loan_rate,product_n,n.limit_over_desire,loan_limit_min.rate,age,gender,credit_score
6,2,62,200,15000000.0,9.3,0,1.0,0.0,0.00,7.0,1.0,1.000000,6.0,9.3,1.0,1.0,15000000.0,42.0,2.0,590.0
7,6,29,265,20000000.0,9.4,0,1.0,0.0,0.00,2.0,1.0,0.333333,6.0,9.4,15.0,3.0,20000000.0,32.0,2.0,630.0
8,6,21,196,23000000.0,12.5,0,2.0,3.1,1.35,2.0,1.0,0.333333,6.0,9.4,15.0,3.0,20000000.0,32.0,2.0,630.0
9,6,64,254,33000000.0,13.2,0,3.0,3.8,0.70,2.0,1.0,0.333333,6.0,9.4,15.0,3.0,20000000.0,32.0,2.0,630.0
10,6,26,80,38000000.0,13.6,0,4.0,4.2,0.40,2.0,1.0,0.333333,6.0,9.4,15.0,3.0,20000000.0,32.0,2.0,630.0


In [63]:
train_2nd.drop(['loan_limit_min.rate','min.loan_rate'],axis=1,inplace=True)

In [65]:
train_2nd.to_csv('/Data2/folder_ti/data/train_2nd.csv',index=False)

In [3]:
path = '/Data2/folder_ti/data/'
train_csv = 'train_2nd.csv'

train_2nd = pd.read_csv(path+train_csv,index_col=0)



/opt/conda/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
train_2nd.reset_index(drop=False,inplace=True)

In [6]:
train_2nd.head()

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,rate_rank,rate_diff_with.min,rate_diff_with.above,purpose,...,houseown_type,desired_amount,rehabilitation,existing_loan_cnt,logdata,diff,diff2,limit_over_desire_product,yd,pred
0,2,59,119,15000000.0,9.3,0,1.0,0.0,0.00,7,...,2,10000000.0,3,11.0,11.0,5000000.0,0.500000,1.0,0.232558,1.0
1,6,28,160,20000000.0,9.4,0,1.0,0.0,0.00,2,...,0,36000000.0,2,4.0,0.0,-16000000.0,-0.444444,0.2,1.000000,1.0
2,6,20,117,23000000.0,12.5,0,2.0,3.1,1.35,2,...,0,36000000.0,2,4.0,0.0,-16000000.0,-0.444444,0.2,1.000000,1.0
3,6,61,152,33000000.0,13.2,0,3.0,3.8,0.70,2,...,0,36000000.0,2,4.0,0.0,-16000000.0,-0.444444,0.2,1.000000,1.0
4,6,25,45,38000000.0,13.6,0,4.0,4.2,0.40,2,...,0,36000000.0,2,4.0,0.0,-16000000.0,-0.444444,0.2,1.000000,1.0


In [7]:
train_2nd.columns

Index(['application_id', 'bank_id', 'product_id', 'loan_limit', 'loan_rate',
       'is_applied', 'rate_rank', 'rate_diff_with.min', 'rate_diff_with.above',
       'purpose', 'past.is_applied', 'm_past.is_applied', 'seq', 'product_n',
       'n.limit_over_desire', 'age', 'gender', 'credit_score', 'yearly_income',
       'income_type', 'company_year', 'employment_type', 'houseown_type',
       'desired_amount', 'rehabilitation', 'existing_loan_cnt', 'logdata',
       'diff', 'diff2', 'limit_over_desire_product', 'yd', 'pred'],
      dtype='object')

In [8]:
train_2nd.drop(['age','gender','income_type','employment_type','houseown_type','yearly_income','existing_loan_cnt'],axis=1,inplace=True)

In [9]:
#nunique = train.nunique()
types = train_2nd.dtypes
target = 'is_applied'
unused_col = ['set','application_id','pred']
#cat_col = ['bank_id','product_id','purpose','income_type','rehabilitation','gender','employment_type','houseown_type']
cat_col = ['bank_id','product_id','purpose','rehabilitation']
categorical_dims =  {}
for col in train_2nd.columns:
    if col in cat_col :
        print(col, train_2nd[col].nunique())
        l_enc = LabelEncoder()
        train_2nd[col] = l_enc.fit_transform(train_2nd[col].values)
        categorical_dims[col] = len(l_enc.classes_)
    elif col not in unused_col+[target]:
        train_2nd.fillna(train_2nd.loc[:, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
features = [ col for col in train_2nd.columns if col not in unused_col+[target]] 
cat_idxs = [ i for i, f in enumerate(features) if f in cat_col]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in cat_col]
#train_shuf = train_shuf.astype({'income_type':'int64','employment_type':'int64','houseown_type':'int64','purpose':'int64'})

bank_id 62
product_id 165
purpose 8
rehabilitation 4


In [10]:
features

['bank_id',
 'product_id',
 'loan_limit',
 'loan_rate',
 'rate_rank',
 'rate_diff_with.min',
 'rate_diff_with.above',
 'purpose',
 'past.is_applied',
 'm_past.is_applied',
 'seq',
 'product_n',
 'n.limit_over_desire',
 'credit_score',
 'company_year',
 'desired_amount',
 'rehabilitation',
 'logdata',
 'diff',
 'diff2',
 'limit_over_desire_product',
 'yd']

In [11]:
data_0 = train_2nd[train_2nd.is_applied == 0]
data_1 = train_2nd[train_2nd.is_applied == 1]
target = 'is_applied'    
if "set" not in data_0.columns:
    data_0["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_0.shape[0],))
if "set" not in data_1.columns:
    data_1["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_1.shape[0],))

/tmp/ipykernel_4599/2215527724.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_0["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_0.shape[0],))
/tmp/ipykernel_4599/2215527724.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_1.shape[0],))


In [12]:
train_2nd = pd.concat([data_0,data_1], ignore_index=True)
train_2nd = train_2nd.sample(frac=1).reset_index(drop=True)

In [13]:
train_indices = train_2nd[train_2nd.set=="train"].index
valid_indices = train_2nd[train_2nd.set=="valid"].index
test_indices = train_2nd[train_2nd.pred==1].index
test_else_indices = train_2nd[train_2nd.pred==0].index

X_train = train_2nd[features].values[train_indices]
y_train = train_2nd[target].values[train_indices]

X_valid = train_2nd[features].values[valid_indices]
y_valid = train_2nd[target].values[valid_indices]

X_test = train_2nd[features].values[test_indices]
y_test = train_2nd[target].values[test_indices]

test_else_set = train_2nd.loc[test_else_indices,:]
test_set = train_2nd.loc[test_indices,:]

In [14]:
import torch
DEVICE = 'cuda:2' if torch.cuda.is_available() else 'cpu'
print("Using {}".format(DEVICE))

params = {'n_d' : 4,
         'n_steps' :5,        # tree에 depth
         'gamma':1.5,
         'n_independent':2,
          'cat_idxs':cat_idxs,
          'cat_dims':cat_dims,
         'n_shared':2,
         'cat_emb_dim':1,
         'lambda_sparse':1e-4,
         'momentum':0.5,
         'clip_value':2.,
         'optimizer_fn':torch.optim.Adam,
         'optimizer_params':dict(lr=2e-2),
         'scheduler_params':{"gamma": 0.95,
                         "step_size": 20},
         'scheduler_fn':torch.optim.lr_scheduler.StepLR,
         'epsilon':1e-15,
         'device_name':DEVICE,
         'mask_type':'sparsemax'}

clf = TabNetClassifier(**params
)
s = time.time()

clf.fit(X_train=X_train, y_train=y_train,eval_set=[(X_valid, y_valid)],
    max_epochs=30 , patience=20,
    batch_size=10240, virtual_batch_size=1280,
    num_workers=0,
    weights=1,
    drop_last=False
)

print('TabNet gpu sec : ',time.time()- s)


pred_valid = clf.predict(X_valid)


f1_valid = f1_score(y_valid, pred_valid) # y_true : 정답 값, y_pred : 예측 값 
print("Valid F1 score: {}".format(f1_valid))
 

from sklearn.metrics import roc_auc_score 
roc_auc = roc_auc_score(y_valid, pred_valid)
print("Valid ROC AUC Score: {}".format(roc_auc))

Using cuda:2


/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda:2
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.41493 | val_0_auc: 0.90266 |  0:01:20s
epoch 1  | loss: 0.38231 | val_0_auc: 0.90984 |  0:02:38s
epoch 2  | loss: 0.36867 | val_0_auc: 0.91337 |  0:03:56s
epoch 3  | loss: 0.35936 | val_0_auc: 0.91865 |  0:05:17s
epoch 4  | loss: 0.3519  | val_0_auc: 0.92103 |  0:06:36s
epoch 5  | loss: 0.35116 | val_0_auc: 0.922   |  0:07:53s
epoch 6  | loss: 0.34539 | val_0_auc: 0.91345 |  0:09:11s
epoch 7  | loss: 0.34731 | val_0_auc: 0.92325 |  0:10:30s
epoch 8  | loss: 0.34245 | val_0_auc: 0.92389 |  0:11:47s
epoch 9  | loss: 0.34148 | val_0_auc: 0.92463 |  0:13:04s
epoch 10 | loss: 0.3394  | val_0_auc: 0.92505 |  0:14:22s
epoch 11 | loss: 0.33914 | val_0_auc: 0.92533 |  0:15:41s
epoch 12 | loss: 0.33869 | val_0_auc: 0.92509 |  0:16:59s
epoch 13 | loss: 0.33839 | val_0_auc: 0.9259  |  0:18:16s
epoch 14 | loss: 0.33726 | val_0_auc: 0.92603 |  0:19:35s
epoch 15 | loss: 0.33677 | val_0_auc: 0.92608 |  0:20:53s
epoch 16 | loss: 0.33642 | val_0_auc: 0.92618 |  0:22:10s
epoch 17 | los

/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TabNet gpu sec :  2609.6403727531433
Valid F1 score: 0.3580792404601059
Valid ROC AUC Score: 0.853156572337846


## 0.34, 0.85

In [15]:
pred_prob_valid = clf.predict_proba(X_valid)

def thres_optimization(threshold, array) :
    pred = np.ceil(array[:,1]-threshold)
    return pred

score_valid = [0]
best_thres_valid = 0
for i in range(1,100) :
    pre_valid = thres_optimization(i/100,pred_prob_valid)
    
    f1_valid = f1_score(y_valid, pre_valid)
    
    score_valid.append(f1_valid)
    
best_thres_valid = score_valid.index(max(score_valid))/100
print(max(score_valid))
print(best_thres_valid)

    

0.4765324595658093
0.82


In [16]:
pred_prob_test = clf.predict_proba(X_test)

def thres_optimization(threshold, array) :
    pred = np.ceil(array[:,1]-threshold)
    return pred

score_test = [0]
best_thres_test = 0
for i in range(1,100) :
    pre_test = thres_optimization(i/100,pred_prob_test)
    
    f1_test = f1_score(y_test, pre_test)
    
    score_test.append(f1_test)
    
best_thres_test = score_test.index(max(score_test))/100
print(max(score_test))
print(best_thres_test)

0.4964737930686575
0.83


In [17]:
test_else_set['pred2'] = 0
test_set['pred2'] = thres_optimization(0.85,pred_prob_test)

In [18]:
final_test = pd.concat([test_else_set,test_set], ignore_index=True)

In [19]:
print('2nd_classification_f1_score : ',f1_score(final_test.is_applied.values, final_test.pred2.values))

2nd_classification_f1_score :  0.4703220347453553


In [20]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(final_test.is_applied.values, final_test.pred2.values))

[[4696658  165833]
 [ 141528  136459]]


## Smote 2nd_classification

In [20]:
from imblearn.over_sampling import SMOTE

smote_sample = SMOTE(random_state=0)
X_train_over,y_train_over = smote_sample.fit_resample(X_train,y_train)

In [21]:
import torch
DEVICE = 'cuda:2' if torch.cuda.is_available() else 'cpu'
print("Using {}".format(DEVICE))

params = {'n_d' : 4,
         'n_steps' :5,        # tree에 depth
         'gamma':1.5,
         'n_independent':2,
          'cat_idxs':cat_idxs,
          'cat_dims':cat_dims,
         'n_shared':2,
         'cat_emb_dim':1,
         'lambda_sparse':1e-4,
         'momentum':0.5,
         'clip_value':2.,
         'optimizer_fn':torch.optim.Adam,
         'optimizer_params':dict(lr=2e-2),
         'scheduler_params':{"gamma": 0.95,
                         "step_size": 20},
         'scheduler_fn':torch.optim.lr_scheduler.StepLR,
         'epsilon':1e-15,
         'device_name':DEVICE,
         'mask_type':'sparsemax'}

clf = TabNetClassifier(**params
)
s = time.time()

clf.fit(X_train=X_train_over, y_train=y_train_over,eval_set=[(X_valid, y_valid)],
    max_epochs=20 , patience=20,
    batch_size=10240, virtual_batch_size=1280,
    num_workers=0,
    weights=1,
    drop_last=False
)

print('TabNet gpu sec : ',time.time()- s)


pred_valid = clf.predict(X_valid)


f1_valid = f1_score(y_valid, pred_valid) # y_true : 정답 값, y_pred : 예측 값 
print("Valid F1 score: {}".format(f1_valid))
 

from sklearn.metrics import roc_auc_score 
roc_auc = roc_auc_score(y_valid, pred_valid)
print("Valid ROC AUC Score: {}".format(roc_auc))

Using cuda:2


/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda:2
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.33782 | val_0_auc: 0.87443 |  0:02:32s
epoch 1  | loss: 0.20629 | val_0_auc: 0.89427 |  0:05:04s
epoch 2  | loss: 0.15115 | val_0_auc: 0.90206 |  0:07:37s
epoch 3  | loss: 0.12415 | val_0_auc: 0.89792 |  0:10:23s
epoch 4  | loss: 0.12376 | val_0_auc: 0.90591 |  0:13:06s
epoch 5  | loss: 0.11828 | val_0_auc: 0.90822 |  0:15:38s
epoch 6  | loss: 0.1126  | val_0_auc: 0.89729 |  0:18:10s
epoch 7  | loss: 0.11675 | val_0_auc: 0.89843 |  0:20:44s
epoch 8  | loss: 0.11173 | val_0_auc: 0.9025  |  0:23:38s
epoch 9  | loss: 0.11051 | val_0_auc: 0.88563 |  0:26:47s
epoch 10 | loss: 0.10906 | val_0_auc: 0.89408 |  0:29:53s
epoch 11 | loss: 0.10761 | val_0_auc: 0.82355 |  0:32:51s
epoch 12 | loss: 0.10593 | val_0_auc: 0.86287 |  0:36:31s
epoch 13 | loss: 0.10443 | val_0_auc: 0.89813 |  0:39:47s
epoch 14 | loss: 0.1047  | val_0_auc: 0.88459 |  0:43:02s
epoch 15 | loss: 0.10338 | val_0_auc: 0.86952 |  0:46:20s
epoch 16 | loss: 0.10434 | val_0_auc: 0.86698 |  0:49:38s
epoch 17 | los

/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TabNet gpu sec :  4711.617521524429
Valid F1 score: 0.40708203031032003
Valid ROC AUC Score: 0.6769016812586578


In [22]:
pred_prob_valid = clf.predict_proba(X_valid)

def thres_optimization(threshold, array) :
    pred = np.ceil(array[:,1]-threshold)
    return pred

score_valid = [0]
best_thres_valid = 0
for i in range(1,100) :
    pre_valid = thres_optimization(i/100,pred_prob_valid)
    
    f1_valid = f1_score(y_valid, pre_valid)
    
    score_valid.append(f1_valid)
    
best_thres_valid = score_valid.index(max(score_valid))/100
print(max(score_valid))
print(best_thres_valid)

0.4350965878317386
0.35


In [23]:
pred_prob_test = clf.predict_proba(X_test)

def thres_optimization(threshold, array) :
    pred = np.ceil(array[:,1]-threshold)
    return pred

score_test = [0]
best_thres_test = 0
for i in range(1,100) :
    pre_test = thres_optimization(i/100,pred_prob_test)
    
    f1_test = f1_score(y_test, pre_test)
    
    score_test.append(f1_test)
    
best_thres_test = score_test.index(max(score_test))/100
print(max(score_test))
print(best_thres_test)

0.4581477959864
0.34


In [28]:
test_else_set['pred2'] = 0
test_set['pred2'] = thres_optimization(0.34,pred_prob_test)

In [29]:
final_test = pd.concat([test_else_set,test_set], ignore_index=True)

In [30]:
print('2nd_classification_f1_score : ',f1_score(final_test.is_applied.values, final_test.pred2.values))

2nd_classification_f1_score :  0.4380195750480084


In [31]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(final_test.is_applied.values, final_test.pred2.values))

[[4636177  226314]
 [ 136568  141419]]


-----------------   

# product_id 전부 포함


---------

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier
from tqdm import tqdm
from sklearn.metrics import f1_score ,roc_auc_score
import time
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

import lightgbm as lgb

In [74]:
path = '/Data2/folder_ti/data/'
train_csv = 'classification1_train.csv'
all_train_csv = 'merge_user_loan.csv'

train = pd.read_csv(path+train_csv,index_col=0)
train = train.astype({'gender':'object'})
all_train = pd.read_csv(path+all_train_csv)



In [72]:
all_train.head()

,application_id,bank_id,product_id,loan_limit,loan_rate
0,576643,54,235,11000000.0,16.5
1,576643,11,118,3000000.0,20.0
2,2136706,42,216,10000000.0,13.5
3,2136706,25,169,22000000.0,15.9
4,2136706,22,221,10000000.0,18.4


In [8]:
train.head()

,application_id,last.loan_insert.time,user_id,is_applied,purpose,past.is_applied,m_past.is_applied,seq,age,gender,credit_score,yearly_income,income_type,company_year,employment_type,houseown_type,desired_amount,rehabilitation,existing_loan_cnt,logdata
83604,248576,2022-05-21 23:41:30,9,0,사업자금,0,0.381271,0,27,1,810,29000000.0,EARNEDINCOME,0,계약직,기타가족소유,15000000.0,2,1,3
396890,1180616,2022-03-24 10:58:38,11,1,생활비,0,0.381271,0,26,1,640,25000000.0,EARNEDINCOME,1,정규직,기타가족소유,2000000.0,-1,1,12
588705,1751522,2022-04-20 10:11:18,11,1,대환대출,1,1.000000,1,26,1,670,25000000.0,EARNEDINCOME,1,정규직,기타가족소유,13000000.0,-1,1,21
507209,1508745,2022-04-18 14:37:11,14,0,대환대출,0,0.381271,0,58,0,650,27000000.0,EARNEDINCOME,2,정규직,전월세,30000000.0,-1,0,8
669164,1990215,2022-03-06 10:46:59,17,0,생활비,0,0.381271,0,30,1,540,50000000.0,EARNEDINCOME,0,계약직,기타가족소유,3000000.0,-1,0,20


In [75]:
all_data_0 = all_train[all_train.is_applied == 0]
all_data_1 = all_train[all_train.is_applied == 1]

In [76]:
all_train = pd.concat([all_data_0,all_data_1], ignore_index=True)

In [77]:
all_train.user_id.nunique()

259328

In [14]:
all_train.iloc[:,:12].head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,insert_time,credit_score
0,576643,2022-05-09 10:54:53,54,235,11000000.0,16.5,0.0,545882,1977.0,1.0,2022-05-09 10:54:51,580.0
1,576643,2022-05-09 10:54:53,11,118,3000000.0,20.0,0.0,545882,1977.0,1.0,2022-05-09 10:54:51,580.0
2,2136706,2022-05-09 10:41:06,42,216,10000000.0,13.5,0.0,558819,1983.0,1.0,2022-05-09 10:41:05,740.0
3,2136706,2022-05-09 10:41:07,25,169,22000000.0,15.9,0.0,558819,1983.0,1.0,2022-05-09 10:41:05,740.0
4,2136706,2022-05-09 10:41:07,22,221,10000000.0,18.4,0.0,558819,1983.0,1.0,2022-05-09 10:41:05,740.0


In [10]:
all_train.columns

Index(['application_id', 'loanapply_insert_time', 'bank_id', 'product_id',
       'loan_limit', 'loan_rate', 'is_applied', 'user_id', 'birth_year',
       'gender', 'insert_time', 'credit_score', 'yearly_income', 'income_type',
       'company_enter_month', 'employment_type', 'houseown_type',
       'desired_amount', 'purpose', 'personal_rehabilitation_yn',
       'personal_rehabilitation_complete_yn', 'existing_loan_cnt',
       'existing_loan_amt'],
      dtype='object')

In [11]:
train.columns

Index(['application_id', 'last.loan_insert.time', 'user_id', 'is_applied',
       'purpose', 'past.is_applied', 'm_past.is_applied', 'seq', 'age',
       'gender', 'credit_score', 'yearly_income', 'income_type',
       'company_year', 'employment_type', 'houseown_type', 'desired_amount',
       'rehabilitation', 'existing_loan_cnt', 'logdata'],
      dtype='object')

In [6]:
all_train.drop(['loanapply_insert_time','is_applied','user_id','birth_year','gender','insert_time','credit_score','yearly_income','income_type',
                'company_enter_month','employment_type','houseown_type','desired_amount','purpose','personal_rehabilitation_yn',
               'personal_rehabilitation_complete_yn','existing_loan_cnt','existing_loan_amt'],axis=1,inplace=True)

In [24]:
all_train.head()

,application_id,bank_id,product_id,loan_limit,loan_rate
0,576643,54,235,11000000.0,16.5
1,576643,11,118,3000000.0,20.0
2,2136706,42,216,10000000.0,13.5
3,2136706,25,169,22000000.0,15.9
4,2136706,22,221,10000000.0,18.4


In [25]:
for col in all_train.columns :
    print('col name & nan : ', col,'/',all_train[col].isnull().sum())

col name & nan :  application_id / 0
col name & nan :  bank_id / 0
col name & nan :  product_id / 0
col name & nan :  loan_limit / 5625
col name & nan :  loan_rate / 5625


In [26]:
all_train.nunique()

application_id    728884
bank_id               62
product_id           178
loan_limit           638
loan_rate            173
dtype: int64

In [7]:
train = pd.merge(all_train, train, on='application_id', how='left')

In [8]:
for col in train.columns :
    print('col name & nan : ', col,'/',train[col].isnull().sum())

col name & nan :  application_id / 0
col name & nan :  bank_id / 0
col name & nan :  product_id / 0
col name & nan :  loan_limit / 5625
col name & nan :  loan_rate / 5625
col name & nan :  last.loan_insert.time / 0
col name & nan :  user_id / 0
col name & nan :  is_applied / 0
col name & nan :  purpose / 0
col name & nan :  past.is_applied / 0
col name & nan :  m_past.is_applied / 0
col name & nan :  seq / 0
col name & nan :  age / 0
col name & nan :  gender / 0
col name & nan :  credit_score / 0
col name & nan :  yearly_income / 0
col name & nan :  income_type / 0
col name & nan :  company_year / 0
col name & nan :  employment_type / 0
col name & nan :  houseown_type / 0
col name & nan :  desired_amount / 0
col name & nan :  rehabilitation / 0
col name & nan :  existing_loan_cnt / 0
col name & nan :  logdata / 0


In [30]:
train.nunique()

application_id           728884
bank_id                      62
product_id                  178
loan_limit                  638
loan_rate                   173
last.loan_insert.time    680426
user_id                  259328
is_applied                    2
purpose                       8
past.is_applied               2
m_past.is_applied          1741
seq                          13
age                          73
gender                        3
credit_score                 81
yearly_income                87
income_type                   6
company_year                 11
employment_type               4
houseown_type                 4
desired_amount               65
rehabilitation                4
existing_loan_cnt            12
logdata                      35
dtype: int64

In [31]:
train.count()

application_id           10270011
bank_id                  10270011
product_id               10270011
loan_limit               10264386
loan_rate                10264386
last.loan_insert.time    10270011
user_id                  10270011
is_applied               10270011
purpose                  10270011
past.is_applied          10270011
m_past.is_applied        10270011
seq                      10270011
age                      10270011
gender                   10270011
credit_score             10270011
yearly_income            10270011
income_type              10270011
company_year             10270011
employment_type          10270011
houseown_type            10270011
desired_amount           10270011
rehabilitation           10270011
existing_loan_cnt        10270011
logdata                  10270011
dtype: int64

In [32]:
train.purpose.value_counts()

생활비       5263934
대환대출      3099662
주택구입       444070
전월세보증금     442551
사업자금       426709
기타         311253
투자         223318
자동차구입       58514
Name: purpose, dtype: int64

In [33]:
train.is_applied.value_counts()

0    5973494
1    4296517
Name: is_applied, dtype: int64

In [9]:
train = train.astype({'bank_id':'object','product_id':'object'})

In [36]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10270011 entries, 0 to 10270010
Data columns (total 24 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   application_id         int64  
 1   bank_id                object 
 2   product_id             object 
 3   loan_limit             float64
 4   loan_rate              float64
 5   last.loan_insert.time  object 
 6   user_id                int64  
 7   is_applied             int64  
 8   purpose                object 
 9   past.is_applied        int64  
 10  m_past.is_applied      float64
 11  seq                    int64  
 12  age                    int64  
 13  gender                 object 
 14  credit_score           int64  
 15  yearly_income          float64
 16  income_type            object 
 17  company_year           int64  
 18  employment_type        object 
 19  houseown_type          object 
 20  desired_amount         float64
 21  rehabilitation         int64  
 22  existing_loan_cn

In [10]:
data_0 = train[train.is_applied == 0]
data_1 = train[train.is_applied == 1]
target = 'is_applied'    
if "set" not in data_0.columns:
    data_0["set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(data_0.shape[0],))
if "set" not in data_1.columns:
    data_1["set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(data_1.shape[0],))

/tmp/ipykernel_37288/2034989029.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_0["set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(data_0.shape[0],))
/tmp/ipykernel_37288/2034989029.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1["set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(data_1.shape[0],))


In [11]:
train = pd.concat([data_0,data_1], ignore_index=True)
train_shuf=train.sample(frac=1).reset_index(drop=True)

In [12]:
cat_col = ['bank_id','product_id','purpose','gender','income_type','employment_type','houseown_type']

In [13]:
cat_trans = dict()
for cat in cat_col :
    if cat in ['gender','bank_id','product_id']:
        continue
    cat_trans[cat] = dict()
    type_ = list(set(train[cat].tolist()))
    for i, ty in enumerate(type_) :
        cat_trans[cat][ty] = i

In [12]:
cat_trans

{'purpose': {'기타': 0,
  '생활비': 1,
  '대환대출': 2,
  '투자': 3,
  '전월세보증금': 4,
  '주택구입': 5,
  '사업자금': 6,
  '자동차구입': 7},
 'income_type': {'PRIVATEBUSINESS': 0,
  'FREELANCER': 1,
  'EARNEDINCOME2': 2,
  'EARNEDINCOME': 3,
  'PRACTITIONER': 4,
  'OTHERINCOME': 5},
 'employment_type': {'계약직': 0, '정규직': 1, '기타': 2, '일용직': 3},
 'houseown_type': {'배우자': 0, '자가': 1, '전월세': 2, '기타가족소유': 3}}

In [14]:
from tqdm import tqdm
for cat in cat_col :
    if cat in ['gender','bank_id','product_id'] :
        continue
    for k in tqdm(cat_trans[cat].keys()) :
        train[cat][train[cat]==k] = cat_trans[cat][k]   

  0%|                                                                                                                       | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_37288/1997422126.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[cat][train[cat]==k] = cat_trans[cat][k]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.40it/s]


In [15]:
train_indices = train[train.set=="train"].index
valid_indices = train[train.set=="valid"].index
test_indices = train[train.set=="test"].index

In [16]:
nunique = train_shuf.nunique()
types = train.dtypes
target = 'is_applied'
unused_col = ['application_id','last.loan_insert.time','user_id','set']
cat_col = ['bank_id','product_id','purpose','gender','income_type','employment_type','houseown_type']
categorical_dims =  {}
for col in train.columns:
    if types[col] == 'object' or col in cat_col :
        if col == ('set' or 'last.loan_insert.time') :
            continue
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        train.fillna(train.loc[:, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
features = [ col for col in train.columns if col not in unused_col+[target]] 
cat_idxs = [ i for i, f in enumerate(features) if f in cat_col]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in cat_col]
#train_shuf = train_shuf.astype({'income_type':'int64','employment_type':'int64','houseown_type':'int64','purpose':'int64'})

bank_id 62
product_id 178
last.loan_insert.time 680426
purpose 8
gender 3
income_type 6
employment_type 4
houseown_type 4


In [16]:
for col in train.columns :
    print('col name & nan : ', col,'/',train[col].isnull().sum())

col name & nan :  application_id / 0
col name & nan :  bank_id / 0
col name & nan :  product_id / 0
col name & nan :  loan_limit / 0
col name & nan :  loan_rate / 0
col name & nan :  last.loan_insert.time / 0
col name & nan :  user_id / 0
col name & nan :  is_applied / 0
col name & nan :  purpose / 0
col name & nan :  past.is_applied / 0
col name & nan :  m_past.is_applied / 0
col name & nan :  seq / 0
col name & nan :  age / 0
col name & nan :  gender / 0
col name & nan :  credit_score / 0
col name & nan :  yearly_income / 0
col name & nan :  income_type / 0
col name & nan :  company_year / 0
col name & nan :  employment_type / 0
col name & nan :  houseown_type / 0
col name & nan :  desired_amount / 0
col name & nan :  rehabilitation / 0
col name & nan :  existing_loan_cnt / 0
col name & nan :  logdata / 0
col name & nan :  set / 0


In [34]:
train.head()

,application_id,bank_id,product_id,loan_limit,loan_rate,last.loan_insert.time,user_id,is_applied,purpose,past.is_applied,...,yearly_income,income_type,company_year,employment_type,houseown_type,desired_amount,rehabilitation,existing_loan_cnt,logdata,set
0,576643,51,150,11000000.0,16.5,498619,545882,0,5,1,...,72000000.0,1,3,0,0,20000000.0,2,7,6,valid
1,576643,10,72,3000000.0,20.0,498619,545882,0,5,1,...,72000000.0,1,3,0,0,20000000.0,2,7,6,valid
2,380021,61,165,42000000.0,6.9,497179,341662,0,0,0,...,50000000.0,1,1,0,2,60000000.0,2,0,9,train
3,380021,58,57,50000000.0,11.4,497179,341662,0,0,0,...,50000000.0,1,1,0,2,60000000.0,2,0,9,test
4,380021,7,17,50000000.0,5.1,497179,341662,0,0,0,...,50000000.0,1,1,0,2,60000000.0,2,0,9,train


In [47]:
features

['bank_id',
 'product_id',
 'loan_limit',
 'loan_rate',
 'purpose',
 'past.is_applied',
 'm_past.is_applied',
 'seq',
 'age',
 'gender',
 'credit_score',
 'yearly_income',
 'income_type',
 'company_year',
 'employment_type',
 'houseown_type',
 'desired_amount',
 'rehabilitation',
 'existing_loan_cnt',
 'logdata']

In [17]:
X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

X_test = train[features].values[test_indices]
y_test = train[target].values[test_indices]

In [18]:
import torch
DEVICE = 'cuda:2' if torch.cuda.is_available() else 'cpu'
print("Using {}".format(DEVICE))

Using cuda:2


In [19]:
train.columns

Index(['application_id', 'bank_id', 'product_id', 'loan_limit', 'loan_rate',
       'last.loan_insert.time', 'user_id', 'is_applied', 'purpose',
       'past.is_applied', 'm_past.is_applied', 'seq', 'age', 'gender',
       'credit_score', 'yearly_income', 'income_type', 'company_year',
       'employment_type', 'houseown_type', 'desired_amount', 'rehabilitation',
       'existing_loan_cnt', 'logdata', 'set'],
      dtype='object')

In [20]:
features

['bank_id',
 'product_id',
 'loan_limit',
 'loan_rate',
 'purpose',
 'past.is_applied',
 'm_past.is_applied',
 'seq',
 'age',
 'gender',
 'credit_score',
 'yearly_income',
 'income_type',
 'company_year',
 'employment_type',
 'houseown_type',
 'desired_amount',
 'rehabilitation',
 'existing_loan_cnt',
 'logdata']

In [20]:
print(list(set(train.application_id.tolist())))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
params = {'n_d' : np.random.choice([4,16,64]),
              'n_a' : np.random.choice([4,16,64]),
             'n_steps' :np.random.choice([3,5,7,10]),        # tree에 depth
             'gamma': np.random.choice([1.3,1.5,1.7]),
             'momentum':np.random.choice([0.3,0.5]),
        #     'cat_emb_dim':np.random.choice([1,16,32,64]),
             'mask_type':np.random.choice(['sparsemax','entmax'])
             }

clf = TabNetClassifier(**params, 
                           n_independent = 2, 
                           n_shared = 2, 
                           lambda_sparse = 1e-4, 
                           cat_idxs = cat_idxs, 
                           cat_dims = cat_dims,
                           cat_emb_dim = 1,
                           clip_value = 2,
                          optimizer_fn = torch.optim.Adam,
                           optimizer_params = {'lr':1e-1},
                           scheduler_params = {"gamma": 0.95,
                                             "step_size": 20},
                           scheduler_fn = torch.optim.lr_scheduler.StepLR,
                           epsilon = 1e-15,
                           device_name = DEVICE
                          )
s = time.time()

clf.fit(X_train=X_train, y_train=y_train,eval_set=[(X_valid, y_valid)],
    max_epochs=10 , patience=5,
    batch_size=10240, virtual_batch_size=1280,
    num_workers=0,
    weights=1,
    drop_last=False
)

print('TabNet gpu sec : ',time.time()- s)


pred_valid = clf.predict(X_valid)
pred_test = clf.predict(X_test)


f1_valid = f1_score(y_valid, pred_valid) # y_true : 정답 값, y_pred : 예측 값 
f1_test = f1_score(y_test, pred_test)
print("Valid F1 score: {}".format(f1_valid))
print("Test F1 score: {}".format(f1_test))
 

from sklearn.metrics import roc_auc_score 
roc_auc = roc_auc_score(y_valid, pred_valid)
roc_auc_t = roc_auc_score(y_test, pred_test) 
print("Valid ROC AUC Score: {}".format(roc_auc))
print("Test ROC AUC Score: {}".format(roc_auc_t))

/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda:2
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.62854 | val_0_auc: 0.71047 |  0:02:41s
epoch 1  | loss: 0.61707 | val_0_auc: 0.71436 |  0:05:19s
epoch 2  | loss: 0.61443 | val_0_auc: 0.71735 |  0:07:58s
epoch 3  | loss: 0.61055 | val_0_auc: 0.71869 |  0:10:36s
epoch 4  | loss: 0.60765 | val_0_auc: 0.7252  |  0:13:13s
epoch 5  | loss: 0.60383 | val_0_auc: 0.72849 |  0:15:52s
epoch 6  | loss: 0.60148 | val_0_auc: 0.73204 |  0:18:32s
epoch 7  | loss: 0.59907 | val_0_auc: 0.73409 |  0:21:09s
epoch 8  | loss: 0.59719 | val_0_auc: 0.73581 |  0:23:45s
epoch 9  | loss: 0.59623 | val_0_auc: 0.73823 |  0:26:23s
Stop training because you reached max_epochs = 10 with best_epoch = 9 and best_val_0_auc = 0.73823


/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TabNet gpu sec :  2232.703307390213
Valid F1 score: 0.6512721102040089
Test F1 score: 0.6513437057991514
Valid ROC AUC Score: 0.6723237257217322
Test ROC AUC Score: 0.6720355008001336


In [23]:
from tqdm import tqdm
print('params : ',params)
pred_prob_valid = clf.predict_proba(X_valid)
pred_prob_test = clf.predict_proba(X_test)

def thres_optimization(threshold, array) :
    pred = np.ceil(array[:,1]-threshold)
    return pred

score_valid = [0]
score_test = [0]
best_thres_valid = 0
best_thres_test = 0
for i in tqdm(range(1,100)) :
    pre_valid = thres_optimization(i/100,pred_prob_valid)
    pre_test = thres_optimization(i/100,pred_prob_test)
    
    f1_valid = f1_score(y_valid, pre_valid)
    f1_test = f1_score(y_test, pre_test)
    
    score_valid.append(f1_valid)
    score_test.append(f1_test)
    
    
    
    
    

params :  {'n_d': 16, 'n_a': 16, 'n_steps': 3, 'gamma': 1.3, 'momentum': 0.3, 'mask_type': 'sparsemax'}


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [01:48<00:00,  1.09s/it]


In [24]:
best_thres_valid = score_valid.index(max(score_valid))/100
best_thres_test = score_test.index(max(score_test))/100
print(max(score_valid))
print(max(score_test))
print(best_thres_valid)
print(best_thres_test)

0.6613939536338085
0.6618892681886068
0.41
0.42


In [25]:
pred_prob_valid = clf.predict_proba(X_valid)
pred_prob_test = clf.predict_proba(X_test)

In [35]:
valid = train.loc[valid_indices,features+[target,'application_id']]
test = train.loc[test_indices,features+[target,'application_id']]
valid['pred'] = thres_optimization(0.41,pred_prob_valid)
test['pred'] = thres_optimization(0.42,pred_prob_test)

In [36]:
f1_score(valid['is_applied'].values, valid['pred'].values)

0.6613939536338085

In [37]:
valid.head()

,bank_id,product_id,loan_limit,loan_rate,purpose,past.is_applied,m_past.is_applied,seq,age,gender,...,company_year,employment_type,houseown_type,desired_amount,rehabilitation,existing_loan_cnt,logdata,is_applied,application_id,pred
0,51,150,11000000.0,16.5,5,1,0.4,5,46,2,...,3,0,0,20000000.0,2,7,6,0,576643,1.0
1,10,72,3000000.0,20.0,5,1,0.4,5,46,2,...,3,0,0,20000000.0,2,7,6,0,576643,1.0
7,50,149,50000000.0,4.0,0,0,0.0,1,32,2,...,1,0,2,60000000.0,2,0,9,0,380021,-0.0
27,60,143,40000000.0,14.4,0,0,0.0,1,32,2,...,1,0,2,60000000.0,2,0,9,0,380021,-0.0
36,37,7,45000000.0,11.1,0,0,0.0,1,32,2,...,1,0,2,60000000.0,2,0,9,0,380021,-0.0


In [41]:
exp = pd.concat([valid,test], ignore_index=True)
app_mean = pd.DataFrame(exp['pred'].groupby(exp['application_id']).mean())

In [52]:
app_mean.head()

,pred
application_id,
6,1.0
14,1.0
20,0.0
26,0.0
28,0.0


In [54]:
app_mean['application_id'] = app_mean.index
print(app_mean.application_id.head())
print(app_mean.application_id.nunique())

application_id
6      6
14    14
20    20
26    26
28    28
Name: application_id, dtype: int64
512762


In [50]:
path = '/Data2/folder_ti/data/'
train_csv = 'classification1_train.csv'
all_train_csv = 'merge_user_loan.csv'

train2 = pd.read_csv(path+train_csv,index_col=0)

In [55]:
train2.application_id.nunique()

728884

In [57]:
train2.reset_index(drop = True, inplace = True)
app_mean.reset_index(drop = True, inplace = True)

In [62]:
train_exp = pd.merge(train2, app_mean, on='application_id', how='left')

In [63]:
for col in train_exp.columns :
    print('col name & nan : ', col,'/',train_exp[col].isnull().sum())

col name & nan :  application_id / 0
col name & nan :  last.loan_insert.time / 0
col name & nan :  user_id / 0
col name & nan :  is_applied / 0
col name & nan :  purpose / 0
col name & nan :  past.is_applied / 0
col name & nan :  m_past.is_applied / 0
col name & nan :  seq / 0
col name & nan :  age / 0
col name & nan :  gender / 0
col name & nan :  credit_score / 0
col name & nan :  yearly_income / 0
col name & nan :  income_type / 0
col name & nan :  company_year / 0
col name & nan :  employment_type / 0
col name & nan :  houseown_type / 0
col name & nan :  desired_amount / 0
col name & nan :  rehabilitation / 0
col name & nan :  existing_loan_cnt / 0
col name & nan :  logdata / 0
col name & nan :  pred / 216122


In [65]:
train_exp.dropna(axis=0, how='any',inplace=True)

In [66]:
for col in train_exp.columns :
    print('col name & nan : ', col,'/',train_exp[col].isnull().sum())

col name & nan :  application_id / 0
col name & nan :  last.loan_insert.time / 0
col name & nan :  user_id / 0
col name & nan :  is_applied / 0
col name & nan :  purpose / 0
col name & nan :  past.is_applied / 0
col name & nan :  m_past.is_applied / 0
col name & nan :  seq / 0
col name & nan :  age / 0
col name & nan :  gender / 0
col name & nan :  credit_score / 0
col name & nan :  yearly_income / 0
col name & nan :  income_type / 0
col name & nan :  company_year / 0
col name & nan :  employment_type / 0
col name & nan :  houseown_type / 0
col name & nan :  desired_amount / 0
col name & nan :  rehabilitation / 0
col name & nan :  existing_loan_cnt / 0
col name & nan :  logdata / 0
col name & nan :  pred / 0


In [68]:
def thres_optim(df, thres) :
    pred = np.ceil(np.array(df['pred'].values)-thres)
    f1 = f1_score(df['is_applied'],pred)
    return f1

In [69]:
score = [0]
best_thres = 0
for i in tqdm(range(1,100)) :
    f1 = thres_optim(train_exp,i/100)
    score.append(f1)
best_thres = score.index(max(score))/100
print(max(score))
print(best_thres)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:31<00:00,  3.16it/s]

0.6450440658434058
0.19


In [70]:
best_thres_valid = score.index(max(score))/100
best_thres_test = score_test.index(max(score_test))/100
print(max(score_valid))
print(max(score_test))
print(best_thres_valid)
print(best_thres_test)

0.6613939536338085
0.6618892681886068
0.41
0.42


AttributeError: 'list' object has no attribute 'lower'